In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wiki-cc-openweb/wiki_small_50k/wiki_small_50k.csv
/kaggle/input/wiki-cc-openweb/openweb_50k (2)/openweb_50k.csv
/kaggle/input/wiki-cc-openweb/cc_news/cc_news.csv


In [1]:
from datasets import load_dataset, concatenate_datasets

In [2]:
wiki_ds = load_dataset("csv", data_files="/kaggle/input/wiki-cc-openweb/wiki_small_50k/wiki_small_50k.csv")
owt_ds = load_dataset("csv", data_files="/kaggle/input/wiki-cc-openweb/openweb_50k (2)/openweb_50k.csv")
cc_news_ds = load_dataset("csv", data_files="/kaggle/input/wiki-cc-openweb/cc_news/cc_news.csv")
train_ds = concatenate_datasets([wiki_ds["train"],owt_ds["train"],cc_news_ds["train"]])

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
from tokenizers import ByteLevelBPETokenizer
special_tokens = ['<s>','<pad>','</s>','<unk>','<mask>']
tokenizer = ByteLevelBPETokenizer()
tokenizer.train_from_iterator([x['text'] for x in train_ds], vocab_size = 10_000,min_frequency =2, special_tokens=special_tokens)


In [4]:
tokenizer.save_model("/kaggle/working/")

['/kaggle/working/vocab.json', '/kaggle/working/merges.txt']

In [26]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("/kaggle/working/")

In [33]:
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, max_length=254)

train_dataset = train_ds.map(tokenize, batched=True, remove_columns=["text"])
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

Map:   0%|          | 0/170824 [00:00<?, ? examples/s]

In [34]:
sample_dataset = train_dataset[:5]

In [38]:
# Real text


sample_dataset = []
for text in train_ds['text']:
    enc = tokenizer(text, truncation=True, max_length=254, return_tensors="pt")
    sample_dataset.append({
        "input_ids": enc["input_ids"].squeeze(0),
        "attention_mask": enc["attention_mask"].squeeze(0)
    })


In [39]:
test_dataset = sample_dataset[:5]

In [11]:
from transformers import (
    RobertaConfig,
    RobertaForMaskedLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)

2025-08-22 15:23:53.571424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755876233.834870      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755876233.909321      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
config = RobertaConfig(
vocab_size = tokenizer.vocab_size,
hidden_size = 256, #Embedding size
num_hidden_layers = 6,#Number of encoder layers
num_attention_heads=4, # Attentions per encoder layer, divisible by 256
intermediate_size = 1024, #FFN layer size
max_position_embeddings=256, #Position Embedding max size = embedding size
type_vocab_size =1,
layer_norm_eps = 1e-5,
initializer_range = 0.02,
pad_token_id = 1,
bos_token_id = 0,
eos_token_id = 2
)
model = RobertaForMaskedLM(config)


In [14]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)


In [ ]:
# 4️⃣ Training arguments
training_args = TrainingArguments(
    output_dir="./checkpoints",
    overwrite_output_dir = False,
    do_train=True,
    save_steps=100,
    logging_steps=100,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    warmup_steps = 100,
    logging_dir = "./logs",
    fp16=False,
    save_total_limit=3,
    save_strategy = "steps",
    report_to=[]
)

# 5️⃣ Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=sample_dataset,
    data_collator=data_collator
)
trainer.train()

Step,Training Loss
100,8.540700
200,7.962200
300,7.561200
400,7.427800
500,7.375600
600,7.336800
700,7.301300
800,7.279600
900,7.270000
1000,7.232700


In [5]:
import os
os.listdir("/kaggle/working/virtual_documents")

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/virtual_documents'

In [ ]:
import json
import matplotlib.pyplot as plt

# Path to your trainer_state.json (inside output_dir)
state_file = "./checkpoints/trainer_state.json"

with open(state_file, "r") as f:
    trainer_state = json.load(f)

# Extract loss and step history
steps = []
losses = []

for log in trainer_state.get("log_history", []):
    if "loss" in log:
        steps.append(log["step"])
        losses.append(log["loss"])

# Plot
plt.figure(figsize=(8, 5))
plt.plot(steps, losses, marker="o")
plt.xlabel("Step")
plt.ylabel("Training Loss")
plt.title("Training Loss over Steps")
plt.grid(True)
plt.show()


In [ ]:
import json
import matplotlib.pyplot as plt

# Path to your trainer_state.json
state_file = "./checkpoints/trainer_state.json"

with open(state_file, "r") as f:
    trainer_state = json.load(f)

# Extract steps, loss, and learning rate
steps = []
losses = []
lrs = []

for log in trainer_state.get("log_history", []):
    if "loss" in log and "learning_rate" in log:
        steps.append(log["step"])
        losses.append(log["loss"])
        lrs.append(log["learning_rate"])

# Plot with dual axes
fig, ax1 = plt.subplots(figsize=(9, 5))

color = "tab:blue"
ax1.set_xlabel("Step")
ax1.set_ylabel("Training Loss", color=color)
ax1.plot(steps, losses, color=color, marker="o", label="Loss")
ax1.tick_params(axis="y", labelcolor=color)
ax1.grid(True)

ax2 = ax1.twinx()  # second y-axis
color = "tab:red"
ax2.set_ylabel("Learning Rate", color=color)
ax2.plot(steps, lrs, color=color, linestyle="--", label="Learning Rate")
ax2.tick_params(axis="y", labelcolor=color)

fig.suptitle("Training Loss and Learning Rate over Steps")
plt.show()
